# Treino dos modelos

Este notebook contêm apenas o pré-processamento e o treino dos modelos. Para o relatório e uma descrição completa do trabalho, acessar o arquivo `relatorio_ex5.ipynb`

In [2]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

TESTING = True

Using TensorFlow backend.


In [3]:
batch_size = 32
num_classes = 7
epochs = 50
if TESTING:
    epochs = 1

 # This will do preprocessing and realtime data augmentation:
datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    zca_epsilon=1e-06,  # epsilon for ZCA whitening
    rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
    # randomly shift images horizontally (fraction of total width)
    width_shift_range=0.1,
    # randomly shift images vertically (fraction of total height)
    height_shift_range=0.1,
    shear_range=0.,  # set range for random shear
    zoom_range=0.,  # set range for random zoom
    channel_shift_range=0.,  # set range for random channel shifts
    # set mode for filling points outside the input boundaries
    fill_mode='nearest',
    cval=0.,  # value used for fill_mode = "constant"
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False,  # randomly flip images
    # set rescaling factor (applied before any other transformation)
    rescale=None,
    # set function that will be applied on each input
    preprocessing_function=None,
    # image data format, either "channels_first" or "channels_last"
    data_format=None,
    # fraction of images reserved for validation (strictly between 0 and 1)
    validation_split=0.0)
   

"""
# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
"""

"\n# The data, split between train and test sets:\n(x_train, y_train), (x_test, y_test) = cifar10.load_data()\nprint('x_train shape:', x_train.shape)\nprint(x_train.shape[0], 'train samples')\nprint(x_test.shape[0], 'test samples')\n\n# Convert class vectors to binary class matrices.\ny_train = keras.utils.to_categorical(y_train, num_classes)\ny_test = keras.utils.to_categorical(y_test, num_classes)\n\nx_train = x_train.astype('float32')\nx_test = x_test.astype('float32')\nx_train /= 255\nx_test /= 255\n"

In [3]:
def create_model(filters, dropout_conv, extra_conv):
    model = Sequential()
    model.add(Conv2D(32, filters, padding='same',
                     input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(32, filters))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    if dropout_conv:
        model.add(Dropout(0.25))

    model.add(Conv2D(64, filters, padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, filters))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    if dropout_conv:
        model.add(Dropout(0.25))
        
    if extra_conv:
        model.add(Conv2D(64, filters, padding='same'))
        model.add(Activation('relu'))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))
    
    return model

In [15]:
train_generator = datagen.flow_from_directory("train/", target_size=(226, 226))

Found 501 images belonging to 1 classes.


In [16]:
x = train_generator.next()

In [17]:
x[0].shape

(32, 226, 226, 3)

In [4]:
def run_and_save(model_name, data_augmentation, score=False, 
                 filters=(3, 3), dropout_conv=True, extra_conv=False):
    # initiate RMSprop optimizer
    model = create_model(filters, dropout_conv, extra_conv)
    opt = keras.optimizers.RMSprop(lr=0.0001, decay=1e-6)

    # Let's train the model using RMSprop
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    print("CHANGE TO FLOW FROM DIR? TODO")
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=-1)

    # Save model and weights
    model_path = model_name
    model.save(model_path)
    print(f"Saved trained model at {model_path}")

    if score:
        # Score trained model.
        scores = model.evaluate(x_test, y_test, verbose=1)
        print('Test loss:', scores[0])
        print('Test accuracy:', scores[1])

In [5]:
%%time
run_and_save("default_model.h5", data_augmentation)

Using real-time data augmentation.
Epoch 1/50
1563/1563 [==============================] - 134s 86ms/step - loss: 1.8479 - acc: 0.3180 - val_loss: 1.5200 - val_acc: 0.4541
Epoch 2/50
1563/1563 [==============================] - 131s 84ms/step - loss: 1.5683 - acc: 0.4276 - val_loss: 1.3744 - val_acc: 0.5013
Epoch 3/50
1563/1563 [==============================] - 128s 82ms/step - loss: 1.4516 - acc: 0.4741 - val_loss: 1.3161 - val_acc: 0.5266
Epoch 4/50
1563/1563 [==============================] - 132s 85ms/step - loss: 1.3779 - acc: 0.5077 - val_loss: 1.2458 - val_acc: 0.5523
Epoch 5/50
1563/1563 [==============================] - 132s 84ms/step - loss: 1.3133 - acc: 0.5291 - val_loss: 1.1547 - val_acc: 0.5940
Epoch 6/50
1563/1563 [==============================] - 131s 84ms/step - loss: 1.2558 - acc: 0.5534 - val_loss: 1.1030 - val_acc: 0.6108
Epoch 7/50
1563/1563 [==============================] - 125s 80ms/step - loss: 1.1987 - acc: 0.5724 - val_loss: 1.0625 - val_acc: 0.6231
Epoch 

In [7]:
#%%time
#run_and_save("extra_conv.h5", data_augmentation,
#            filters=(5,5), extra_conv=True)

Using real-time data augmentation.
Epoch 1/50
1563/1563 [==============================] - 241s 154ms/step - loss: 1.8418 - acc: 0.3122 - val_loss: 1.5355 - val_acc: 0.4327
Epoch 2/50
1563/1563 [==============================] - 240s 154ms/step - loss: 1.5266 - acc: 0.4400 - val_loss: 1.4411 - val_acc: 0.4865
Epoch 3/50
1563/1563 [==============================] - 240s 154ms/step - loss: 1.3846 - acc: 0.5007 - val_loss: 1.2609 - val_acc: 0.5468
Epoch 4/50
1563/1563 [==============================] - 240s 154ms/step - loss: 1.2775 - acc: 0.5412 - val_loss: 1.1235 - val_acc: 0.6014
Epoch 5/50
1563/1563 [==============================] - 240s 153ms/step - loss: 1.2005 - acc: 0.5703 - val_loss: 1.0075 - val_acc: 0.6455
Epoch 6/50
1563/1563 [==============================] - 240s 153ms/step - loss: 1.1217 - acc: 0.6007 - val_loss: 1.1103 - val_acc: 0.6225
Epoch 7/50
1563/1563 [==============================] - 239s 153ms/step - loss: 1.0629 - acc: 0.6239 - val_loss: 1.0351 - val_acc: 0.6423